<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/Natural%20Language%20Processing/Introduction_to_NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in Tensorflow

In [1]:
!nvidia-smi

Thu Feb  9 17:07:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-09 17:07:05--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-09 17:07:06 (101 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import plot_loss_curves,unzip_data,create_tensorboard_callback,compare_historys

## Getting a text Dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-02-09 17:07:10--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.128, 172.217.194.128, 74.125.130.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2023-02-09 17:07:10 (101 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

## Visualizing Dataset

In [6]:
import pandas as pd
train_dataset = pd.read_csv("/content/train.csv")

In [7]:
train_dataset

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
train_dataset_shuffled = train_dataset.sample(frac = 1,random_state = 42)


In [9]:
train_dataset_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [10]:
test_data  = pd.read_csv("/content/test.csv")

In [11]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [12]:
train_dataset.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
import random
random_index = random.randint(0,len(train_dataset)-5)
for row in train_dataset_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(text,target)

Alhaji Putin is far from being a good person sha. At least I had front row seat to his complete obliteration of Ibeto cement a competitor. 0
A random hole just broke out in the street ?? http://t.co/dWU8QqYs0v 0
Help yourself or those you love who suffer from self-esteem wounds. You can today! http://t.co/tu6ScRSXVG http://t.co/iDhj4JBQ05 0
U.S. Forest Service firefighter David Ruhl 38 died in the 'Frog Fire' in the Modoc National Forest. He had been temporarily reassigned 1
http://t.co/ETkd58Un8n - Cleveland Heights Shaker Heights fight blight: The House Next Door http://t.co/LRelVrm06w 0


## Split Data into Training and Validation Sets

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences,val_sentences,train_labels,val_labels = train_test_split(
    train_dataset_shuffled['text'].to_numpy(),
    train_dataset_shuffled['target'].to_numpy(),
    test_size = 0.1,
    random_state = 42
    )

In [16]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

## Tokenization and Embedding

### Text Vectorization (Tokenization)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens = 10000,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = None,
    pad_to_max_tokens = True,
  )


In [18]:
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

14.901036345059115

### Text Vecotrizer Variables

In [19]:
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)

In [20]:
text_vectorizer.adapt(train_sentences)

In [21]:
sample_sentence = "Flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
random_sentences = random.choice(train_sentences)
text_vectorizer([random_sentences])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   8,   29, 3045,   14,  608,   13, 1067,    4,    2,  127,  943,
          16,   23,    3,  297]])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
top_5_words

['', '[UNK]', 'the', 'a', 'in']

## Embedding Layer

In [24]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
      output_dim = 128,
      input_length = max_length
                                   
)
embedding

In [25]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
embedded_sentence = embedding(text_vectorizer(random_sentence))
print(embedded_sentence)

@BroseidonRex @dapurplesharpie I skimmed through twitter and missed this body bagging.
tf.Tensor(
[[-0.02724484  0.03714091 -0.02836943 ... -0.03115805  0.02237533
  -0.04402329]
 [-0.02724484  0.03714091 -0.02836943 ... -0.03115805  0.02237533
  -0.04402329]
 [-0.03088179  0.00696503  0.04065892 ... -0.02384867  0.0285409
   0.04944719]
 ...
 [-0.0258649  -0.0224664   0.03517495 ...  0.0208034   0.03370311
  -0.03622372]
 [-0.0258649  -0.0224664   0.03517495 ...  0.0208034   0.03370311
  -0.03622372]
 [-0.0258649  -0.0224664   0.03517495 ...  0.0208034   0.03370311
  -0.03622372]], shape=(15, 128), dtype=float32)


## Model 0 : Naive Bayes (Baseline)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), #convert words to numbers,
    ("clf",MultinomialNB()), #model the text,
])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
base_line_score = model_0.score(val_sentences,val_labels)

In [28]:
base_line_score

0.7926509186351706

In [29]:
base_line_predict = model_0.predict(val_sentences)
base_line_predict[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating functions to compare model experiments

In [30]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)*100
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = "weighted")
  model_results = {
      "accuracy":model_accuracy,
      "precision":model_precision,
      "recall":model_recall,
      "f1":model_f1
  }
  return model_results

In [31]:
base_line_results = calculate_results(val_labels,base_line_predict)
base_line_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: Simple Dense Model

In [33]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

In [44]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")

In [45]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [46]:
model_1.compile(loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy'])

In [47]:
model_1_history = model_1.fit(
    x = train_sentences,y = train_labels,
    epochs = 5,
    validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_1_dense")]
    )

Saving TensorBoard log files to: model_logs/model_1_dense/20230209-172458
Epoch 1/5
215/215 [==============================] - 2s 5ms/step - loss: 0.5147 - accuracy: 0.7968 - val_loss: 0.4925 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3262 - accuracy: 0.8821 - val_loss: 0.4723 - val_accuracy: 0.7992
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2563 - accuracy: 0.9061 - val_loss: 0.4852 - val_accuracy: 0.7966
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2100 - accuracy: 0.9238 - val_loss: 0.5027 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1746 - accuracy: 0.9393 - val_loss: 0.5222 - val_accuracy: 0.8005


In [48]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.5222 - accuracy: 0.8005


[0.522177517414093, 0.8005249500274658]

In [49]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 3ms/step


(762, 1)

In [52]:
model_1_pred_probs[10]

array([0.04094055], dtype=float32)

In [53]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [54]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [56]:
model_1_results = calculate_results(y_true = val_labels,y_pred = model_1_preds)
model_1_results

{'accuracy': 80.0524934383202,
 'precision': 0.8018367833195843,
 'recall': 0.800524934383202,
 'f1': 0.7991414898011219}

## Visualizing Learned Embeddings

In [57]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [60]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [58]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]

In [61]:
embed_weights.shape

(10000, 128)

In [62]:
embed_weights

array([[-0.1105111 ,  0.24386404, -0.06131501, ..., -0.07562449,
         0.24938092, -0.09743926],
       [-0.05828946,  0.15509187, -0.07914229, ..., -0.09016415,
         0.12291872, -0.12485715],
       [-0.05634741, -0.04156812, -0.07612676, ..., -0.06036295,
        -0.04086918, -0.06818093],
       ...,
       [-0.01055761,  0.03920205, -0.01721575, ..., -0.03774003,
        -0.03887228, -0.00991591],
       [-0.06476884, -0.03519952, -0.00887774, ...,  0.0170569 ,
        -0.03668934, -0.06046719],
       [-0.09301209, -0.02949233,  0.11723753, ...,  0.09225851,
        -0.08759896, -0.1249655 ]], dtype=float32)